In [ ]:
# Install required packages.
!pip install -q torch-scatter -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
!pip install -q torch-sparse -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
!pip install -q torch-geometric

# Helper function for visualization.
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

def visualize(h, color):
    z = TSNE(n_components=2).fit_transform(out.detach().cpu().numpy())

    plt.figure(figsize=(10,10))
    plt.xticks([])
    plt.yticks([])

    plt.scatter(z[:, 0], z[:, 1], s=70, c=color, cmap="Set2")
    plt.show()

    

#I. Defining perturbations of the adjacency

In [ ]:
#### Teh dropout function already has a lot of the modalities that we want
import numpy as np
import torch
import torch_geometric as tg
from torch_sparse import coalesce

def filter_adj(row, col, edge_attr, mask):
    return row[mask], col[mask], None if edge_attr is None else edge_attr[mask]

def drop_edges(edge_index, n_nodes, edge_attr=None, p=0.5, force_undirected=False,
                num_nodes=None
               ):
    r"""Randomly drops edges from the adjacency matrix
    :obj:`(edge_index, edge_attr)` with probability :obj:`p` using samples from
    a Bernoulli distribution.

    Args:
        edge_index (LongTensor): The edge indices.
        edge_attr (Tensor, optional): Edge weights or multi-dimensional
            edge features. (default: :obj:`None`)
        p (float, optional): Dropout probability. (default: :obj:`0.5`)
        force_undirected (bool, optional): If set to :obj:`True`, will either
            drop or keep both edges of an undirected edge.
            (default: :obj:`False`)
        num_nodes (int, optional): The number of nodes, *i.e.*
            :obj:`max_val + 1` of :attr:`edge_index`. (default: :obj:`None`)
        training (bool, optional): If set to :obj:`False`, this operation is a
            no-op. (default: :obj:`True`)
    """

    if p < 0. or p > 1.:
        raise ValueError('Dropout probability has to be between 0 and 1, '
                         'but got {}'.format(p))

    if p == 0.0:
        return edge_index, edge_attr

    N = num_nodes
    row, col = edge_index

    if force_undirected:
        row, col, edge_attr = filter_adj(row, col, edge_attr, row < col)
    mask = edge_index.new_full((row.size(0), ), 1 - p, dtype=torch.float)
    mask = torch.bernoulli(mask).to(torch.bool)

    row, col, edge_attr = filter_adj(row, col, edge_attr, mask)

    if force_undirected:
        edge_index = torch.stack(
            [torch.cat([row, col], dim=0),
             torch.cat([col, row], dim=0)], dim=0)
    else:
        edge_index = torch.stack([row, col], dim=0)

    return edge_index, edge_attr   

In [ ]:
def drop_nb_edges(edge_index, n_edges2delete, edge_attr=None, force_undirected=False,
                num_nodes=None, connectedness_constraint = True, 
                drop_random=True, drop_strategy = "degree"
               ):
    r"""Randomly drops a number edges from the adjacency matrix, whilst ensuring
    that the graph remains connected
    :obj:`(edge_index, edge_attr)` with probability :obj:`p` using samples from
    a Bernoulli distribution.

    Args:
        edge_index (LongTensor): The edge indices.
        edge_attr (Tensor, optional): Edge weights or multi-dimensional
            edge features. (default: :obj:`None`)
        p (float, optional): Dropout probability. (default: :obj:`0.5`)
        force_undirected (bool, optional): If set to :obj:`True`, will either
            drop or keep both edges of an undirected edge.
            (default: :obj:`False`)
        num_nodes (int, optional): The number of nodes, *i.e.*
            :obj:`max_val + 1` of :attr:`edge_index`. (default: :obj:`None`)
        training (bool, optional): If set to :obj:`False`, this operation is a
            no-op. (default: :obj:`True`)
    """


    if n_edges2delete == 0.0:
        return edge_index, edge_attr

    if  num_nodes == None: N = edge_index.max().item()
    else:  N = num_nodes
    row, col = edge_index
    degree_seq = [torch.sum(row == k).tolist() for k in np.arange(num_nodes)]

    if force_undirected:
        row, col, edge_attr = filter_adj(row, col, edge_attr, row < col)
    
    #mask = edge_index.new_full((row.size(0), ), 1 - p, dtype=torch.float)
    mask = edge_index.new_full((row.size(0), ), True, dtype=torch.bool)
    if drop_random:
      delete_edges = np.random.choice(np.arange(len(row)), size=n_edges2delete )
    else:
      #### drop edges with higher probability if the nodes is well connected
      p = [ min([degree_seq[col[i]], degree_seq[col[i]]]) for i in np.arange(len(row))]
      delete_edges = np.random.choice(np.arange(len(row)), size=n_edges2delete, p = p )
      
    mask[delete_edges] = False
  
    row2, col2, edge_attr2 = filter_adj(row, col, edge_attr, mask)

    if force_undirected:
        edge_index = torch.stack(
            [torch.cat([row2, col2], dim=0),
             torch.cat([col2, row2], dim=0)], dim=0)
    else:
        edge_index = torch.stack([row2, col2], dim=0)
    ##### Unsures that the matrix remains connected

    if connectedness_constraint:
      row2, col2 = edge_index
      degree_seq2 = [torch.sum(row2 == k).tolist() for k in np.arange(num_nodes
                                                                      )]
      unconnected_nodes = np.where(np.array(degree_seq2) < 1)[0]
      #print(unconnected_nodes)
      if len(unconnected_nodes)>0:
          for i in unconnected_nodes:
            initial_edge = np.hstack([col[np.where(row == i)[0]],row[np.where(col == i)[0]]])
            uu = np.random.choice(initial_edge)
            edge_index = torch.hstack([edge_index, torch.tensor([[i, uu ],[uu, i ]]) ] )
    
    #row2, col2 = edge_index
    #degree_seq2 = [torch.sum(row2 == k).tolist() for k in np.arange(num_nodes)]
    #unconnected_nodes = np.where(np.array(degree_seq2) < 1)[0]
    #print(unconnected_nodes)
    return edge_index, edge_attr

In [ ]:
#### TO DO: add perturbations targeted at some location of the graph (using tg.utils.k_hop_subgraph)

# II. Defining random graph structures

In [ ]:
#### TO DO: generate synthetic graphs
#### Let's use networkx
import networkx as nx
def SyntheticData(num_nodes, n_classes, graph_generator_type="planted_partition",
                  p_in=0.5, p_out=0.1):
  x= torch.ones((num_nodes, n_classes +10)) ### Node features
  nn. int(num_nodes/n_classes)
  if graph_generator_type=="planted_partition":
    G = nx.planted_partition_graph(nn, n_classes,
                                   p_in, p_out, seed=42
                                   )
    labels = torch.tensor(np.concatenate([[u] * 10  for u in np.arange(7)]), dtype=int)
  dd = tg.utils.convert.from_networkx(G)
  edge_index= dd.edge_index ### Edge Index
  edge_attr=None ### Edge features
  y = labels ### Node labels
  return tg.data.Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=y) 



In [ ]:
tg.

# III. Assessing the performance on the graphs

### a. Defining the GCN pipeline

In [ ]:
from torch_geometric.nn import GCNConv


class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(dataset.num_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, dataset.num_classes)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return x

model = GCN(hidden_channels=16)
print(model)




In [ ]:
from IPython.display import Javascript  # Restrict height of output cell.
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))

def GCN_pipeline(data, edge_index):
    model = GCN(hidden_channels=16)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    criterion = torch.nn.CrossEntropyLoss()
    def train():
          model.train()
          optimizer.zero_grad()  # Clear gradients.
          out = model(data.x, edge_index)  # Perform a single forward pass.
          loss = criterion(out[data.train_mask], data.y[data.train_mask])  # Compute the loss solely based on the training nodes.
          loss.backward()  # Derive gradients.
          optimizer.step()  # Update parameters based on gradients.
          return loss

    def test():
          model.eval()
          out = model(data.x, edge_index)
          pred = out.argmax(dim=1)  # Use the class with highest probability.
          test_correct = pred[data.test_mask] == data.y[data.test_mask]  # Check against ground-truth labels.
          test_acc = int(test_correct.sum()) / int(data.test_mask.sum())  # Derive ratio of correct predictions.
          return test_acc


    for epoch in range(1, 201):
        loss = train()
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')
    
    return test()

### b. Defining the MLP pipeline (for comparison)


In [ ]:
from IPython.display import Javascript  # Restrict height of output cell.
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))


def MLP_pipeline(data):
  model = MLP(hidden_channels=16)
  criterion = torch.nn.CrossEntropyLoss()  # Define loss criterion.
  optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)  # Define optimizer.

  def train():
        model.train()
        optimizer.zero_grad()  # Clear gradients.
        out = model(data.x)  # Perform a single forward pass.
        loss = criterion(out[data.train_mask], data.y[data.train_mask])  # Compute the loss solely based on the training nodes.
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        return loss

  def test():
        model.eval()
        out = model(data.x)
        pred = out.argmax(dim=1)  # Use the class with highest probability.
        test_correct = pred[data.test_mask] == data.y[data.test_mask]  # Check against ground-truth labels.
        test_acc = int(test_correct.sum()) / int(data.test_mask.sum())  # Derive ratio of correct predictions.
        return test_acc

  for epoch in range(1, 201):
      loss = train()
      print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

  return test()

### c. Run experiments


In [ ]:
##### 
b = 0 
mlp_res = []
gnn_res =[]
while b < 100:
  edge_index2,_ = drop_nb_edges(data.edge_index, 600, edge_attr=None, force_undirected=True,
                num_nodes=data.num_nodes, connectedness_constraint = True)
  
  mlp_res += [MLP_pipeline(data)]
  gnn_res += [GCN_pipeline(data, edge_index2)]
  b += 1